Task: Get a list of fantasy book IDs to loop through and find shelf data for, then find shelves to do with diverse books, etc.

Need to: 

-get list of books tagged fantasy (genre tag...)

-scrape book IDs from Fantasy pages

-come up with shelf keywords to isolate

In [27]:
from goodreads import client
from lxml import html
import requests
from collections import Counter

gc = client.GoodreadsClient("Ha1oKI3R0fqeApxCJIcQ", "wbHQF5APtAgwKrY0kQ9gFSxyVEqt0kEqWwi3HUf0t7A")

# Thank you internet for helping with scraping
# http://docs.python-guide.org/en/latest/scenarios/scrape/

In [16]:
# set a up a loop to get the ID numbers from all the items that come up when I search "fantasy" 
# in the genre tag on goodreads. I'm using the search results instead of the popular fantasy books page
# so I can have more data
# This section is also when I learn the value of testing on a smaller data set first. This takes a bit to run.

search_page = 'http://www.goodreads.com/search?page=1&q=fantasy&search%5Bfield%5D=genre&search_type=books&tab=books&utf8=%E2%9C%93'
page_num = 1
book_IDs = []
for page in range(100):
    fantasy_books = requests.get(search_page)
    tree = html.fromstring(fantasy_books.content)
    book_IDs.append(tree.xpath("//form[@class='hiddenShelfForm']/input[@name='book_id']/@value"))
    page_num += 1
    search_page = 'http://www.goodreads.com/search?page='+(str(page_num))+'&q=fantasy&search%5Bfield%5D=genre&search_type=books&tab=books&utf8=%E2%9C%93'

loop through the list of lists (book_IDs) and collect the shelf data for each ID number using goodreads api.

things I can do with the shelf data:

identify titles on "diverse books" type shelves

count the number of fantasy books with "divesre books" type shelving

Maybe best to create a dictionary for this type of task.

In [18]:
books_and_shelves = {} # key = book title, value = list of popular shelves

for book_list in book_IDs:
    for booknum in book_list:
        book = gc.book(booknum)
        books_and_shelves[book.title] = book.popular_shelves

So now I have my book titles and their popular shelves in a dictionary. I want to find the intersection of fantasy titles and diverse books, but I need to figure out what those shelves are. To start that, I want to just get a list of unique shelves.

In [49]:
# get just the values from the dictionary and create a counter object

allshelves = list(books_and_shelves.values())

# loop over the list of lists to get a single list of shelves

oneshelflist = []
for shelf_list in allshelves:
    oneshelflist.extend(shelf_list)
    
# The counter wouldn't work on the above list. Stack Overflow discussion suggests it may be 
# because the items aren't strings (?) -- I don't really understand why, but the below works.

shelf_strings = []
for shelf in oneshelflist:
    shelf_strings.append(str(shelf))
    
unique_shelves = list(Counter(shelf_strings).keys())


Now that I have a list of unique shelves, I want to find the ones that are related to diversity. To simplify matters, I'm going to look for shelves with "diverse" and "diversity" in them. (There's a regex angle here, but at the moment this is simpler.)

In [54]:
diverse_books = []
for shelf in unique_shelves:
    if "divers" in shelf:
        diverse_books.append(shelf)
        
# This only gets me a list of 6 shelves, but for the time being, let's go with that.

In [60]:
# So now I want to go back to my original dictionary and find the books that have those shelves.
# Then, I can look at what other shelves intersect those, and get an idea of how many fantasy books
# are being labeled "diverse"

for title, shelflist in books_and_shelves.items(): # loop over the dictionary
    for shelf in shelflist:
        for tag in diverse_books:
            if tag in shelf:
                print(title)
    

TypeError: argument of type 'GoodreadsShelf' is not iterable

Imma try that again. Same problem as above--the list of shelves isn't a list of strings. That's stupid.

In [67]:
# change the shelflists into list of strings

for title, shelves in books_and_shelves.items():
    list_of_shelves = list(shelves)
    shelves_str = []
    for shelf in list_of_shelves:
        shelves_str.append(str(shelf))
    books_and_shelves[title] = shelves_str

In [72]:
# NOW loop through to get the titles. Yass it worked.

diverse_titles = []
for title, shelves in books_and_shelves.items():
    for shelf in shelves:
        for tag in diverse_books:
            if tag in shelf:
                diverse_titles.append(title)

In [82]:
# So now I can see what other shelves those books are on
# First, get the list of unique titles
unique_titles = list(Counter(diverse_titles).keys())
diverse_shelves = []
for title, shelves in books_and_shelves.items():
    if title in unique_titles:
        diverse_shelves.extend(shelves)
print(Counter(diverse_shelves))


Counter({'favorites': 61, 'to-read': 61, 'books-i-own': 61, 'fantasy': 60, 'owned': 60, 'to-buy': 60, 'fiction': 59, 'library': 59, 'currently-reading': 58, 'ebook': 58, 'diversity': 58, 'read-in-2016': 55, 'owned-books': 55, 'wish-list': 55, 'young-adult': 54, 'read-in-2017': 54, 'kindle': 54, 'ebooks': 54, 'ya': 52, 'favourites': 52, 'adventure': 48, 'magic': 47, 'sci-fi-fantasy': 47, 'read-2017': 46, 'series': 45, 'read-2016': 45, 'dnf': 45, 'maybe': 45, 'audiobook': 44, 'teen': 44, 'reviewed': 44, 'audiobooks': 43, '2017-reads': 43, 'paranormal': 43, 'romance': 42, 'audio': 42, 'fantasy-sci-fi': 41, 'ya-fantasy': 40, '2016-reads': 40, 'ya-fiction': 40, 'did-not-finish': 39, 'supernatural': 38, 'tbr': 37, 'i-own': 35, 'need-to-buy': 34, 'ya-lit': 32, 'arc': 32, 'read-in-2015': 30, 'science-fiction': 30, 'library-books': 30, 'sci-fi': 29, 'young-adult-fiction': 29, 'e-book': 29, 'lgbt': 28, 'book-club': 28, 'read-in-english': 28, '4-stars': 28, 'ya-books': 27, 'abandoned': 27, 'high-